In [ ]:
# Energy Consumption Forecasting with XGBoost (Improved & Explained)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from pandas.tseries.holiday import USFederalHolidayCalendar

# Plot settings
color_pal = sns.color_palette()
plt.style.use('fivethirtyeight')

# 1. Load the dataset (change path as needed)
df = pd.read_csv("PJME_hourly.csv")  # Assuming this is the file name
df['Datetime'] = pd.to_datetime(df['Datetime'])
df = df.set_index('Datetime')

# 2. Plot the data
df['PJME_MW'].plot(figsize=(15, 5), title='Energy Consumption (MW)')
plt.show()

# 3. Train/Test Split
train = df.loc[df.index < '2015-01-01']
test = df.loc[df.index >= '2015-01-01']

# 4. Feature Engineering (with lag and rolling features)
def create_features(df):
    df = df.copy()
    df['hour'] = df.index.hour
    df['dayofweek'] = df.index.dayofweek
    df['quarter'] = df.index.quarter
    df['month'] = df.index.month
    df['year'] = df.index.year
    df['dayofyear'] = df.index.dayofyear
    df['dayofmonth'] = df.index.day
    df['weekofyear'] = df.index.isocalendar().week.astype(int)

    # Lag features
    df['lag1'] = df['PJME_MW'].shift(1)
    df['lag24'] = df['PJME_MW'].shift(24)
    df['lag168'] = df['PJME_MW'].shift(168)  # One week lag

    # Rolling mean
    df['rolling24'] = df['PJME_MW'].shift(1).rolling(window=24).mean()
    df['rolling168'] = df['PJME_MW'].shift(1).rolling(window=168).mean()

    # Holiday feature
    cal = USFederalHolidayCalendar()
    holidays = cal.holidays(start=df.index.min(), end=df.index.max())
    df['is_holiday'] = df.index.normalize().isin(holidays).astype(int)

    return df

train = create_features(train)
test = create_features(test)

# Drop rows with NaN values caused by lag/rolling features
train.dropna(inplace=True)
test.dropna(inplace=True)

# 5. Define features and target
FEATURES = ['dayofyear', 'hour', 'dayofweek', 'quarter', 'month', 'year',
            'lag1', 'lag24', 'lag168', 'rolling24', 'rolling168', 'is_holiday']
TARGET = 'PJME_MW'

X_train = train[FEATURES]
y_train = train[TARGET]
X_test = test[FEATURES]
y_test = test[TARGET]

# 6. Train the XGBoost model
reg = xgb.XGBRegressor(n_estimators=1000,
                       early_stopping_rounds=50,
                       objective='reg:squarederror',
                       max_depth=5,
                       learning_rate=0.05,
                       n_jobs=-1)
reg.fit(X_train, y_train,
        eval_set=[(X_train, y_train), (X_test, y_test)],
        verbose=100)

# 7. Forecast
preds = reg.predict(X_test)
test['prediction'] = preds

# 8. Evaluate performance
rmse = np.sqrt(mean_squared_error(test[TARGET], test['prediction']))
print(f"\n✅ RMSE on test set: {rmse:.2f} MW")

# 9. Plot predictions vs actual
ax = test[TARGET].plot(figsize=(15, 5), title='Actual vs Predicted Energy Consumption')
test['prediction'].plot(ax=ax)
plt.legend(['Actual', 'Prediction'])
plt.show()

# 10. Feature Importance
fi = pd.DataFrame(data=reg.feature_importances_,
                  index=reg.feature_names_in_,
                  columns=['importance'])
fi.sort_values('importance').plot(kind='barh', title='Feature Importance')
plt.show()

# 11. Look at worst predicted days
test['error'] = np.abs(test[TARGET] - test['prediction'])
test['date'] = test.index.date
worst_days = test.groupby('date')['error'].mean().sort_values(ascending=False).head(5)
print("\n🔍 Worst predicted days:")
print(worst_days)

# The End — Model, Predictions, Evaluation & Feature Improvements Done!